# 简单模块化

说到最简单的模块化方式，你可以把函数、类、常量拆分到不同的文件，把它们放在同一个文件夹，然后使用 from your_file import function_name, class_name 的方式调用。之后，这些函数和类就可以在文件内直接使用了。


In [1]:
# utils_define.py

def get_sum(a, b):
    return a + b


In [2]:
# class_utils.py

class Encoder(object):
    def encode(self, s):
        return s[::-1]

class Decoder(object):
    def decode(self, s):
        return ''.join(reversed(list(s)))


In [6]:
# main.py

from materials.modularization.class_utils.utils_define import get_sum
from materials.modularization.class_utils import *

print(get_sum(1, 2))

encoder = Encoder()
decoder = Decoder()

print(encoder.encode('abcde'))
print(decoder.decode('edcba'))


3
edcba
abcde


我们来看这种方式的代码：get_sum() 函数定义在 utils.py，Encoder 和 Decoder 类则在 class_utils.py，我们在 main 函数直接调用 from import ，就可以将我们需要的东西 import 过来。

虽然这个方法很简单，但是所有文件都堆在一个文件夹下也并不是办法。

于是，可以尝试建立一些子文件

In [1]:
# utils_define/getsum.py

def get_sum(a, b):
    return a + b


# utils_define/class_utils.py

class Encoder(object):
    def encode(self, s):
        return s[::-1]

class Decoder(object):
    def decode(self, s):
        return ''.join(reversed(list(s)))

# src/sub_main.py

import sys
sys.path.append("..")

from utils_define.class_utils  import *

encoder = Encoder()
decoder = Decoder()

print(encoder.encode('abcde'))
print(decoder.decode('edcba'))


ModuleNotFoundError: No module named 'utils_define'

而这一次，我们的文件结构是下面这样的：

.
├── utils_define
│   ├── getsum.py
│   └── class_utils.py
├── src
│   └── sub_main.py
└── main.py


main.py 调用子目录的模块时，只需要使用 . 代替 / 来表示子目录，utils_define.getsum 表示 utils_define 子文件夹下的 getsum.py 模块就行。

那如果我们想调用上层目录呢？注意，`sys.path.append("..")` 表示将当前程序所在位置向上提了一级，之后就能调用 utils 的模块了。

同时要注意一点，import 同一个模块只会被执行一次，这样就可以防止重复导入模块出现问题。而且需要避免与官方或第三方包重名。

最后我想再提一下版本区别。你可能在许多教程中看到过这样的要求：我们还需要在模块所在的文件夹新建一个 `__init__.py`，内容可以为空，也可以用来表述包对外暴露的模块接口。不过，事实上，这是 Python 2 的规范。在 Python 3 规范中，`__init__.py` 并不是必须的，很多教程里没提过这一点，或者没讲明白，我希望你还是能注意到这个地方。

# 项目模块化

路径：
在 Linux 系统中，每个文件都有一个绝对路径，以 / 开头，来表示从根目录到叶子节点的路径，例如 /home/ubuntu/Desktop/my_project/test.py，这种表示方法叫作绝对路径。

另外，对于任意两个文件，我们都有一条通路可以从一个文件走到另一个文件，例如 /home/ubuntu/Downloads/example.json。再如，我们从 test.py 访问到 example.json，需要写成 '../../Downloads/example.json'，其中 .. 表示上一层目录。这种表示方法，叫作相对路径。

**通常，一个 Python 文件在运行的时候，都会有一个运行时位置，最开始时即为这个文件所在的文件夹。**这个运行路径以后可以被改变。运行 sys.path.append("..") ，则可以改变当前 Python 解释器的位置。

**但一般而言并不推荐，固定一个确定路径对大型工程来说是非常必要的**。相对路径不是一个很好的选择，因为代码可能会迁移，相对位置会使得重构既不雅观，也容易出错。因此，在大型工程中尽可能使用绝对位置是第一要义。**对于一个独立的项目，所有的模块的追寻方式，最好从项目的根目录开始追溯，这叫做相对的绝对路径**。

在做项目的时候，虽然你不可能把全世界的代码都放到一个文件夹下，但是类似模块化的思想还是要有的——那就是以项目的根目录作为最基本的目录，所有的模块调用，都要通过根目录一层层向下索引的方式来 import。



In [ ]:
.
├── proto
│   ├── mat.py
├── utils
│   └── mat_mul.py
└── src
    └── main.py


In [ ]:
# proto/mat.py

class Matrix(object):
    def __init__(self, data):
        self.data = data
        self.n = len(data)
        self.m = len(data[0])


# utils_define/mat_mul.py

from proto.mat import Matrix

def mat_mul(matrix_1: Matrix, matrix_2: Matrix):
    assert matrix_1.m == matrix_2.n
    n, m, s = matrix_1.n, matrix_1.m, matrix_2.m
    result = [[0 for _ in range(n)] for _ in range(s)]
    for i in range(n):
        for j in range(s):
            for k in range(m):
                result[i][k] += matrix_1.data[i][j] * matrix_2.data[j][k]

    return Matrix(result)


# src/main.py

from proto.mat import Matrix
from utils_define.mat_mul import mat_mul


a = Matrix([[1, 2], [3, 4]])
b = Matrix([[5, 6], [7, 8]])

print(mat_mul(a, b).data)

########## 输出 ##########

[[19, 22], [43, 50]]


假设根目录为'e:\\git\\learning\\python\\materials\\modularization\\project_modulariztion',可以使用`sys.path[0]`,在运行程序之前设置根目录。这样，你的 import 接下来肯定就畅通无阻了。但这显然不是最佳解决方案，把绝对路径写到代码里，是我非常不推荐的方式（你可以写到配置文件中，但找配置文件也需要路径寻找，于是就会进入无解的死循环）。

还有一种方法，可以修改PYTHONHOME进行设置。假如你是在一个虚拟运行环境（事实上，我们提倡，对于每一个项目来说，最好要有一个独立的运行环境来保持包和模块的纯净性），在一个 Virtual Environment 里，你能找到一个文件叫 activate，在这个文件的末尾，填上下面的内容：`export PYTHONPATH="/home/ubuntu/workspace/your_projects"`.这样，每次你通过 activate 激活这个运行时环境的时候，它就会自动将项目的根目录添加到搜索路径中去。


# 神奇的 if __name__ == '__main__'

Python 是脚本语言，和 C++、Java 最大的不同在于，不需要显式提供 main() 函数入口。如果你有 C++、Java 等语言经验，应该对 main() {} 这样的结构很熟悉吧？

不过，既然 Python 可以直接写代码，if __name__ == '__main__' 这样的写法，除了能让 Python 代码更好看（更像 C++ ）外，还有什么好处吗？

In [ ]:
.
├── utils.py
├── utils_with_main.py
├── main.py
└── main_2.py


In [ ]:
# utils.py

def get_sum(a, b):
    return a + b

print('testing')
print('{} + {} = {}'.format(1, 2, get_sum(1, 2)))


In [ ]:
# utils_with_main.py

def get_sum(a, b):
    return a + b

if __name__ == '__main__':
    print('testing')
    print('{} + {} = {}'.format(1, 2, get_sum(1, 2)))


In [ ]:
# main.py

from utils import get_sum

print('get_sum: ', get_sum(1, 2))

########## 输出 ##########

testing
1 + 2 = 3
get_sum: 3


In [ ]:
# main_2.py

from utils_with_main import get_sum

print('get_sum: ', get_sum(1, 2))

########## 输出 ##########

get_sum_2: 3


import 在导入文件的时候，会自动把所有暴露在外面的代码全都执行一遍。因此，如果你要把一个东西封装成模块，又想让它可以执行的话，你必须将要执行的代码放在 `if __name__ == '__main__'`下面。

为什么呢？其实，`__name__` 作为 Python 的魔术内置参数，本质上是模块对象的一个属性。我们使用 import 语句时，`__name__` 就会被赋值为该模块的名字，自然就不等于 `__main__`了。

# 总结
1. 通过绝对路径和相对路径，我们可以 import 模块；
2. 在大型工程中模块化非常重要，模块的索引要通过绝对路径来做，而绝对路径从程序的根目录开始；
3. 记着巧用 if __name__ == '__main__'来避开 import 时执行。

